## Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None

In [4]:
RANDOM = 12345

## Загрузка данных. Обработка

In [5]:
arc = pd.read_csv('/datasets/final_steel/data_arc.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/final_steel/data_arc.csv'

In [5]:
arc.sample(5,random_state=RANDOM)

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
4557,984,2019-06-08 12:00:37,2019-06-08 12:02:18,0.392892,0.301553
5274,1144,2019-06-14 02:02:26,2019-06-14 02:04:31,0.572077,0.380522
1668,359,2019-05-17 01:03:38,2019-05-17 01:07:32,0.963486,0.659530
10682,2323,2019-07-30 18:57:21,2019-07-30 19:01:13,0.927218,0.762333
4321,927,2019-06-06 12:45:46,2019-06-06 12:50:09,1.122471,0.807470


In [6]:
arc.columns = ['key','begin','end','power_i','power_o']

In [7]:
arc.shape

(14876, 5)

In [8]:
arc.dtypes

key          int64
begin       object
end         object
power_i    float64
power_o    float64
dtype: object

In [9]:
arc['begin'] = pd.to_datetime(arc['begin'],format='%Y-%m-%d')
arc['end'] = pd.to_datetime(arc['end'],format='%Y-%m-%d')
arc['delta'] = (arc['end']-arc['begin']).dt.total_seconds()
arc['power_io'] = arc['power_i']/arc['power_o']

In [10]:
arc.dtypes

key                  int64
begin       datetime64[ns]
end         datetime64[ns]
power_i            float64
power_o            float64
delta              float64
power_io           float64
dtype: object

In [11]:
arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 7 columns):
key         14876 non-null int64
begin       14876 non-null datetime64[ns]
end         14876 non-null datetime64[ns]
power_i     14876 non-null float64
power_o     14876 non-null float64
delta       14876 non-null float64
power_io    14876 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(1)
memory usage: 813.7 KB


## Сводная таблица по партиям

In [18]:
arc_svod = (arc.groupby('key')
 .agg(time_first = ('begin','min'),     #Время первого нагрева
      time_last = ('end','max'),        #Время последнего нагрева
      q_arc = ('key','count'),          #Количество запусков 
      power_io = ('power_io','mean'),   #Среднее соотношение
      time_heat = ('delta','sum')       #Суммарное время нагрева, секунд
     )
).reset_index()

In [19]:
arc_svod['time_full'] = (arc_svod['time_last']-arc_svod['time_first']).dt.total_seconds() #Общая длительность

In [20]:
arc_svod.sample(5,random_state=RANDOM)

,key,time_first,time_last,q_arc,power_io,time_heat,time_full
1415,1431,2019-06-23 20:08:58,2019-06-23 20:56:32,6,1.408322,640.0,2854.0
916,930,2019-06-06 18:56:52,2019-06-06 19:15:24,3,1.294630,859.0,1112.0
1670,1687,2019-07-02 13:56:02,2019-07-02 14:39:05,6,1.543998,1311.0,2583.0
686,698,2019-05-29 04:39:33,2019-05-29 04:48:01,2,1.020349,364.0,508.0
2951,2977,2019-08-23 12:19:56,2019-08-23 12:47:18,4,1.520188,626.0,1642.0


In [21]:
arc_svod[['time_full','time_heat','q_arc','power_io']].describe()

,time_full,time_heat,q_arc,power_io
count,3214.000000,3214.000000,3214.000000,3214.000000
mean,1898.537648,794.579029,4.628500,1.357452
std,1265.105252,332.448622,1.608647,0.132824
min,57.000000,57.000000,1.000000,0.676106
25%,1260.250000,571.000000,4.000000,1.292892
50%,1661.000000,770.000000,4.000000,1.370550
75%,2285.750000,983.000000,6.000000,1.438292
max,23485.000000,4189.000000,16.000000,1.782164
